# Storing skymaps

## Read raw skymap files

In [ ]:
import pickle
from pathlib import Path
import math
import numpy as np
from lsst.sphgeom import Box, ConvexPolygon, LonLat, UnitVector3d
import yaml
from skymap_to_poly_coords import load_pickle_skymap


In [ ]:

home_dir = Path.home() 
raw_skymaps_dir = home_dir / "skymap-to-poly-coords" / "tests" / "data" / "raw_skymaps"
skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(skymap_path)
lsst_skymap

## A note on rings sky map pixelization 
*(in html comment in this cell)*
<!---## Rings sky map pixelization
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.--->

## Storage options


We would like to support both inner and outer polygons for the tracts.  

Tracts are arranged in "rings", which span horizontal regions of the sky.

🔹 Inner polys:
- represent the exact boundaries of a given tract
- do not overlap

🔹 Outer polys:
- may (will?) overlap with adjacent tracts' outer polys
- are akin to margins in HATS catalogs

## Option 1: Two YAML files (inner polys and outer polys) that store each tracts' corners

#### Inner polys:
- Store per-tract: `ra_min`, `ra_max`, `dec_min`, `dec_max`
- Comes directly from `getRaDecRange(tract_index)`
- Requires 4 floats per tract, so for ~19k tracts:
  4 × 8 B × 19,000 ≈ ~600 KB (before YAML overhead; ~1–1.5 MB total)
- However, could compress to a `.npz` of around ~0.5–0.8 MB

#### Outer polys:
- todo

#### ✅ Pros
- Easy to understand and verify
- Doesn’t require any LSST WCS machinery
- Reading is fast + LSST-free

#### ❌ Cons
- Slightly more disk usage than an ultra-compressed model
- Less precise because of floating points?

### Write inner_poly and outer_poly files

In [4]:
from skymap_to_poly_coords import write_polygons_ra_dec

skymap_out_dir = "/sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/"
inner_poly_path = Path(skymap_out_dir) / "inner_polys.yaml"
outer_poly_path = Path(skymap_out_dir) / "outer_polys.yaml"

write_polygons_ra_dec(lsst_skymap, inner_poly_path, inner=True)
write_polygons_ra_dec(lsst_skymap, outer_poly_path, inner=False)

## Option 2: Reconstruct everything using projection + geometry

What you'd need to store:  

- Global:
  - `projection: "TAN"`
  - `pixel_scale_deg`: (or arcsec/pixel)
  - `tract_width`, `tract_height`: in pixels
  - `overlap_deg`: or overlap in pixels (need this for inner bounds)

- Per-tract:
  - `tract_id`
  - `center_ra`, `center_dec`
  - `rotation` (optional, usually 0)
  - `flipX` (optional, for WCS handedness)

Then, you reconstruct inner and outer polys by:
- Creating a fake `SkyWcs` from these params
- Converting pixel corners of `[0,0]` → `[width,height]` to sky coordinates
- Optionally applying overlap trimming in pixel space

✅ Pros
- Super compact: ~2 floats + a few constants per tract
- Elegant reuse of LSST projection machinery
- One unified format

❌ Cons
- Requires LSST-style WCS math to reconstruct accurately
- Introduces fragility: assumptions about projection type and tract uniformity
- Slightly harder to read/debug